In [2]:
import rasterio
from osgeo import gdal
from osgeo import ogr

### EDA original Geotiff

In [ ]:
org_ds = gdal.Open('cdl_2022_original.tiff')
print(org_ds.GetGeoTransform())
print(org_ds.GetProjection())

### Make individual rasters from each attribute in vector data

In [ ]:
reference_raster = '/home/kenny/repos/flavortown/cdl/cdl_ca_crop/cdl_2022_test.tiff'
ref_ds = gdal.Open(reference_raster)
print(ref_ds.RasterXSize)
print(ref_ds.RasterYSize)

In [7]:
def create_geotiff(input_file, output_file, attribute_name, attribute_dtype, attribute_colorinterp, reference_raster):
    # Open the FlatGeoBuf file
    data_source = ogr.Open(input_file)

    if data_source is None:
        print("Error: Could not open the FlatGeoBuf file.")
        return

    # Get the layer
    layer = data_source.GetLayer()

    # Open the reference raster to get its geotransform and spatial reference
    ref_ds = gdal.Open(reference_raster)
    if ref_ds is None:
        print("Error: Could not open the reference raster.")
        return

    # Create a raster GeoTIFF file with the same extent, resolution, and pixel alignment as the reference raster
    raster_ds = gdal.GetDriverByName('GTiff').Create(output_file, ref_ds.RasterXSize, ref_ds.RasterYSize, 1, attribute_dtype, options=["COMPRESS=DEFLATE", "BIGTIFF=YES"])

    # Set the geotransform using the reference raster
    raster_ds.SetGeoTransform(ref_ds.GetGeoTransform())

    # Set the projection
    raster_ds.SetProjection(ref_ds.GetProjection())

    # Rasterize the attribute to the GeoTIFF file
    band = raster_ds.GetRasterBand(1)
    gdal.RasterizeLayer(raster_ds, [1], layer, options=["ATTRIBUTE=" + attribute_name])

    # Set ColorInterp to something other than "gray" (e.g., "Red", "Green", "Blue", "Palette", etc.)
    band.SetColorInterpretation(attribute_colorinterp)  # Change GCI_RedBand to the desired interpretation

    # Close datasets
    raster_ds = None
    data_source = None
    ref_ds = None

# Replace 'input_file' with the path to your FlatGeoBuf file
input_file = '/home/kenny/repos/flavortown/cdl/cdl_ca_crop/2022/ca_crop_2022.fgb'

# Replace 'output_directory' with the directory where you want to save the GeoTIFF files
output_directory = '/home/kenny/repos/flavortown/cdl/cdl_ca_crop/'

# Replace 'attribute_names' with the names of the attributes you want to rasterize
attribute_names = {
    'yr_planted': {"dtype": gdal.GDT_Int32, "colorinterp": gdal.GCI_GrayIndex},
    'crop_id1': {"dtype": gdal.GDT_Byte, "colorinterp": gdal.GCI_PaletteIndex},
    'crop_id2': {"dtype": gdal.GDT_Byte, "colorinterp": gdal.GCI_PaletteIndex},
    'crop_id3': {"dtype": gdal.GDT_Byte, "colorinterp": gdal.GCI_PaletteIndex},
    'crop_id4': {"dtype": gdal.GDT_Byte, "colorinterp": gdal.GCI_PaletteIndex}
    }

# Replace 'reference_raster' with the path to your existing raster file
reference_raster = '/home/kenny/repos/flavortown/cdl/cdl_ca_crop/cdl_2022_test.tiff'

for attribute_name in attribute_names.keys():
    attribute_dtype = attribute_names[attribute_name]['dtype']
    attribute_colorinterp = attribute_names[attribute_name]['colorinterp']
    output_file = f"{output_directory}{attribute_name}.tif"
    create_geotiff(input_file, output_file, attribute_name, attribute_dtype, attribute_colorinterp, reference_raster)

print("Rasterization completed.")


/home/kenny/miniconda3/envs/DS/lib/python3.12/site-packages/osgeo/ogr.py:593: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Rasterization completed.


***

### Stack Geotiffs and build multi-band raster

In [ ]:
import rasterio

file_list = [
    #'cdl_2022_test.tiff',
    'yr_planted.tif',
    'crop_id1.tif',
    'crop_id2.tif',
    'crop_id3.tif',
    'crop_id4.tif'
]

# Read metadata of first file
with rasterio.open(file_list[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count=len(file_list), compress='deflate', bigtiff='yes')

# Create the stack dataset
with rasterio.open('stack.tif', 'w', **meta) as dst:
    for id, layer in enumerate(file_list, start=1):
        with rasterio.open(layer) as src1:
            # Read all bands and write to the corresponding band in the output file
            for band_idx in range(1, src1.count + 1):
                band_data = src1.read(band_idx)
                dst.write(band_data, indexes=id)


# file_list = [
#     'cdl_2022_test.tiff',
#     'yr_planted.tif',
#     'crop_id1.tif',
#     'crop_id2.tif',
#     'crop_id3.tif',
#     'crop_id4.tif'
# ]

# # Read metadata of first file
# with rasterio.open(file_list[0]) as src0:
#     meta = src0.meta

# # Update meta to reflect the number of layers
# meta.update(count = len(file_list))

# # Read each layer and write it to stack
# with rasterio.open('stack.tif', 'w', **meta) as dst:
#     for id, layer in enumerate(file_list, start=1):
#         with rasterio.open(layer) as src1:
#             dst.write_band(id, src1.read(1))